In [7]:
import sys
sys.path.insert(0, './src')  # 或者绝对路径：/full/path/to/your_project
from src.xmcdata import *
from model import seq2seqParams, Seq2SeqModel
from dataset import  load_xmc_seq2seq_dataset_from_disk
import os


In [8]:
dataset_name = 'eurlex-4k'
model_name = 'bart'
params_path = f'./params/{dataset_name}/{model_name}.json'
data_dir = f"xmc-base/{dataset_name}"

In [9]:
train_dataset, val_dataset = load_xmc_seq2seq_dataset_from_disk(data_dir)
args = seq2seqParams.load_config(params_path)
model =  Seq2SeqModel(args)

Loading datasets from xmc-base/eurlex-4k


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

/root/.pyenv/versions/3.10.13/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
from collections import defaultdict

def find_duplicates(dataset, column_name="text"):
    value_to_indices = defaultdict(list)

    for i, val in enumerate(dataset[column_name]):
        value_to_indices[val].append(i)

    duplicates = {k: v for k, v in value_to_indices.items() if len(v) > 1}

    for val, idxs in duplicates.items():
        print(f"Duplicate entry: {val!r}, at indices: {idxs}")

# 使用
find_duplicates(train_dataset, column_name="input")


Duplicate entry: 'commiss regul ec decemb lai market year revis product estim ungin cotton result provision reduct guid price commiss european commun regard treati establish european commun regard act access greec protocol cotton regard council regul ec product aid cotton indent articl thereof articl commiss regul ec august lai detail rule appli cotton aid scheme revis product estim ungin cotton refer subparagraph articl regul ec result provision reduct guid price establish decemb market year concern articl regul ec revis product estim establish take account progress harvest revis estim fix basi data market year subparagraph articl regul ec decemb start market year amount advanc determin base revis product estim view recent figur quantiti supervis storag notifi member state accord articl regul ec uncertainti situat regard greek product safeti margin percentag increas greec spain portug appli market year provision reduct guid price calcul accord articl regul ec replac actual product rev

In [10]:
task = 'train'
if task == 'train':
    output_dir = f"output/{dataset_name}/{model_name}"
    trainer, _ = model.gen_train(train_dataset, val_dataset,output_dir)
else:
    output_dir = f"output/{dataset_name}/{model_name}"
    model.model.from_pretrained(output_dir)

/root/.pyenv/versions/3.10.13/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training model facebook/bart-base with output directory output/eurlex-4k/bart
Training arguments: seq2seqParams(model_name_or_path='facebook/bart-base', max_input_length=256, max_output_length=64, num_train_epochs=10, batch_size=4, learning_rate=1e-05, use_fp16=True, logging_steps=20, weight_decay=0.01, warmup_ratio=0.1, use_tensorboard=True, num_beams=5)


Epoch,Training Loss,Validation Loss
1,2.007800,1.702371
2,1.339800,1.116499
3,1.150200,0.926652
4,0.889700,0.825710
5,0.934500,0.756031
6,0.799100,0.717083
7,0.667900,0.691451
8,0.593300,0.671302
9,0.627700,0.663786
10,0.616400,0.658711


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

In [ ]:
train_dataset

In [11]:
X_tst_text = load_texts(data_dir+"/X.tst.txt")
#prefix = 'Summary this paragraph using unstemmed words: '
#X_tst_text = [prefix + text for text in X_tst_text]

In [12]:
model.predict(X_tst_text,batch_size=8,max_input_length = args.max_input_length,max_output_length=128,num_beams=args.num_beams,output_path=data_dir)


Generating:   0%|          | 0/484 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/root/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torch/nn/modules/linear.py:125: UserWarning: gemm_and_bias error: CUBLAS_STATUS_INVALID_VALUE when calling cublasLtMatmul with transpose_mat1 1 transpose_mat2 0 m 768 n 520 k 768 mat1_ld 768 mat2_ld 768 result_ld 768 abType 0 cType 0 computeType 68 scaleType 0. Will attempt to recover by calling unfused cublas path. (Triggered internally at /pytorch/aten/src/ATen/cuda/CUDABlas.cpp:1730.)
  return F.linear(input, self.weight, self.bias)
Generating:   0%|          | 1/484 [00:01<09:15,  1.15s/it]/root/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torch/nn/modules/linear.py:125: UserWarning: gemm_and_bias error: CUBLAS_STATUS_INVALID_VALUE when calling cublasLtMatmul with transpose_mat1 1 transpose_mat2 0 m 768 n 496 k 768 mat1_ld 768 

KeyboardInterrupt: 